In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pickle

# Veriyi yükle
df = pd.read_csv('veri.csv')

# 'hasat_tarihi' sütununu datetime tipine dönüştür
df['hasat_tarihi'] = pd.to_datetime(df['hasat_tarihi'])

# Kategorik özellikleri one-hot encoding ile dönüştür
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[['boyut', 'renk']]).toarray()
encoded_df = pd.DataFrame(encoded_features)
df = pd.concat([df, encoded_df], axis=1)
df.drop(['boyut', 'renk'], axis=1, inplace=True)

# Tarih bilgisinden özellikler çıkar
df['hasat_ayi'] = df['hasat_tarihi'].dt.month
df['hasat_gunu'] = df['hasat_tarihi'].dt.day
df.drop('hasat_tarihi', axis=1, inplace=True)

# Özellikleri ve hedefi ayır
X = df.drop('fiyat', axis=1)
y = df['fiyat']

# Veriyi eğitim ve test kümelerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sütun adlarını stringe dönüştür
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

# Sayısal özellikleri ölçeklendir
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# X_train verilerinin ortalamasını ve standart sapmasını kaydet
X_train_mean = scaler.mean_
X_train_scale = scaler.scale_

# Ortalama ve standart sapmayı pickle ile kaydet
with open('scaler_params.pkl', 'wb') as f:
    pickle.dump((X_train_mean, X_train_scale), f)

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'SVR': SVR(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'MLPRegressor': MLPRegressor()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)  # squared parametresi olmadan
    results[name] = rmse

# Döngüden sonra en iyi modeli bul
best_model_name = min(results, key=results.get)
best_model = models[best_model_name]
print(f"En iyi model: {best_model_name} (RMSE: {results[best_model_name]:.2f})")

import joblib
joblib.dump(best_model, 'eniyi.joblib')

En iyi model: Ridge (RMSE: 1.46)


c:\Users\senad\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['eniyi.joblib']